# HW4, DATSCI W261
Team: Kuan Lin, Alejandro J. Rojas, Ricardo Barrera<br/>
Emails: kuanlin@ischool.berkeley.edu, ale@ischool.berkeley.edu, ricardofrank@ischool.berkeley.edu<br/>
Time of Initial Submission: 8:00 AM PST, Thursday, Feb 12, 2016<br/>
W261-1, Spring 2016 Week 4 Homework

# HW 4.0.
What is MrJob? How is it different to Hadoop MapReduce? What are the mappint_init, mapper_final(), combiner_final(), reducer_final() methods? When are they called?
<p>
MrJob is a python object-oriented framework that let's you define classes that integrates Hadoop MapReduce in a very elegant and clean structure so you can write and run MapReduce jobs. It contains methods that leverage Hadoop streaming to accomplish MapReduce taks. An MrJob class lets you define steps that you want your routine to follow. This python program comes with a variery of methods that make it simple to implement MapReduce, among them: mappint_init() -> to initialize mapper values mapper_final() -> returns key,value tuple where value is 1 combiner_final() -> returns key,value tuple where value is local sum reducer_final() -> returns key,value tuple where value is total sum
</p>

# HW 4.1
What is serialization in the context of MrJob or Hadoop? When it used in these frameworks? What is the default serialization mode for input and outputs for MrJob?
<p>
MrJob does not take any input in binary. It accepts either text or json which makes it slower that running Hadoop streaming directly. So after receiving its input it converts it into ninary to do all processing. This is inefficient in terms of processing time but teh framework gives the flexibility to quickly write and run easily readable code which makes it an ideal solution to write and run MapReduce jobs.
</p>

# HW 4.2
Recall the Microsoft logfiles data from the async lecture. The logfiles are described are located at:<br/> https://kdd.ics.uci.edu/databases/msweb/msweb.html<br/>
http://archive.ics.uci.edu/ml/machine-learning-databases/anonymous/ <br/>
This dataset records which areas (Vroots) of www.microsoft.com each user visited in a one-week timeframe in Feburary 1998.
Here, you must preprocess the data on a single node (i.e., not on a cluster of nodes) from the format:
<pre>
C,"10001",10001   #Visitor id 10001
V,1000,1          #Visit by Visitor 10001 to page id 1000
V,1001,1          #Visit by Visitor 10001 to page id 1001
V,1002,1          #Visit by Visitor 10001 to page id 1002
C,"10002",10002   #Visitor id 10001
V
Note: #denotes comments
</pre>
to the format:
<pre>
V,1000,1,C, 10001
V,1001,1,C, 10001
V,1002,1,C, 10001
</pre>
Write the python code to accomplish this.

In [7]:
source_file = 'anonymous-msweb.data'
output_file = 'reformated_webdata.csv'

writer = open(output_file, 'w')
current_visitor = None
for line in open(source_file, 'r'):
    line = line.strip()
    if line == '' or (not line.startswith("C,") and not line.startswith("V")): continue
    lineArr = line.split(',')
    if lineArr[0] == 'C':
        current_visitor = lineArr[2]
    elif current_visitor != None:
        writer.write(line + ',C,' + current_visitor + '\n')
writer.close()

In [9]:
# push the transformed file to HDFS
!hdfs dfs -mkdir /w261/hw4/
!hdfs dfs -put reformated_webdata.csv /w261/hw4/

# HW 4.3:
Find the 5 most frequently visited pages using MrJob from the output of 4.2 (i.e., transfromed log file).

In [61]:
%%writefile top_pages.py
from mrjob.job import MRJob
from mrjob.protocol import RawValueProtocol
from mrjob.step import MRStep

class TopPages(MRJob):
    #OUTPUT_PROTOCOL = RawValueProtocol
    
    def steps(self):
        # two step process: first step calculate total visits for each page.
        # second step ranks the pages using hadoop's shuffle and sort.
        return [
            MRStep(mapper=self.process_input, reducer=self.aggregate_page_counts),
            MRStep(reducer=self.rank_pages, jobconf={
                    'stream.num.map.output.key.fields': 2,
                    'mapreduce.partition.keypartitioner.options': '-k1,1',
                    'mapred.output.key.comparator.class': 'org.apache.hadoop.mapred.lib.KeyFieldBasedComparator',
                    'mapred.text.key.comparator.options': "-k2,2nr",
                    'mapred.reduce.tasks': 1
                })
        ]
    
    def process_input(self, _, line):
        # simpily emits page counts as per raw file
        lineArr = line.strip().split(',')
        page_id = lineArr[1]
        visit_counts = int(lineArr[2])
        yield page_id, visit_counts
    
    def aggregate_page_counts(self, page_id, visit_counts):
        #calculate counts per page
        yield page_id, sum(visit_counts)
        
    def rank_pages(self, page_id, total_visits):
        #page should have come in sorted by hadoop, so just printing them out.
        yield page_id, max(total_visits)
        
if __name__ == '__main__':
    TopPages.run()

Overwriting top_pages.py


In [62]:
!hdfs dfs -rm -r /w261/hw4/4_3;
!python top_pages.py -r hadoop hdfs:///w261/hw4/reformated_webdata.csv --output-dir hdfs:///w261/hw4/4_3/ --no-output
#!python top_pages.py reformated_webdata.csv.backup

16/02/06 04:18:05 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /w261/hw4/4_3
no configs found; falling back on auto-configuration
no configs found; falling back on auto-configuration
creating tmp directory /tmp/top_pages.root.20160206.041806.848882
writing wrapper script to /tmp/top_pages.root.20160206.041806.848882/setup-wrapper.sh
Using Hadoop version 2.6.0
Copying local files into hdfs:///user/root/tmp/mrjob/top_pages.root.20160206.041806.848882/files/

PLEASE NOTE: Starting in mrjob v0.5.0, protocols will be strict by default. It's recommended you run your job with --strict-protocols or set up mrjob.conf as described at https://pythonhosted.org/mrjob/whats-new.html#ready-for-strict-protocols

HADOOP: packageJobJar: [] [/usr/lib/hadoop/lib/hadoop-streaming-2.6.0.jar] /tmp/streamjob1514787184897643950.jar tmpDir=null
HADOOP: Connecting to ResourceManager at /0.0.0.0:8032
HADOOP: Connecting to ResourceMa

In [63]:
# top 5 most visited pages:
!hdfs dfs -cat /w261/hw4/4_3/part* | head -5

"1008"	10836
"1034"	9383
"1004"	8463
"1018"	5330
"1017"	5108


# HW 4.4:
Find the most frequent visitor of each page using MrJob and the output of 4.2 (i.e., transfromed log file). In this output please include the webpage URL, webpageID and Visitor ID.

In [6]:
# need to first extract out the urls, will be used to lookup url in reducer
import csv
source_file = 'anonymous-msweb.data'
output_file = 'url_data.csv'
writer = open(output_file, 'w')

for line in open(source_file, 'r'):
    line = line.strip()
    if line == '':
        continue
    if line.startswith('A,'):
        lineArr = list(csv.reader(line.splitlines(), delimiter=',', quotechar='"'))[0]
        writer.write(','.join([lineArr[1], lineArr[4].replace('/', '')]) + '\n')
writer.close()

In [7]:
!cat url_data.csv | head -10

1287,autoroute
1288,library
1289,masterchef
1297,centroam
1215,developer
1279,msgolf
1239,msconsult
1282,home
1251,referencesupport
1121,magazine


In [23]:
%%writefile top_visitor.py
from mrjob.job import MRJob
from mrjob.protocol import RawValueProtocol
from mrjob.step import MRStep

class TopVisitor(MRJob):
    
    OUTPUT_PROTOCOL = RawValueProtocol
    
    def steps(self):
        # two step processes: firs step to calculate visit counts per user and page combination
        # second step to rank user by their visit counts within each page
        return [
            MRStep(mapper=self.process_visits, reducer=self.aggregate_visits),
            MRStep(reducer_init=self.load_url_info, reducer=self.find_most_freq_visitor)
        ]
    
    def process_visits(self, _, line):
        # emiting visit counts per user and page combination
        lineArr = line.strip().split(',')
        page_id = lineArr[1]
        visit_counts = int(lineArr[2])
        visitor_id = lineArr[4]
        yield (page_id, visitor_id), visit_counts 
        
    def aggregate_visits(self, page_visitor_id, visit_counts):
        # calculate how many visits from each visitor on the particular page
        yield page_visitor_id[0], (page_visitor_id[1], sum(visit_counts))
        
    def load_url_info(self):
        # loading up URL dictionary to used for lookup in reducer
        self.URLs = {}
        for line in open('url_data.csv', 'r'):
            lineArr = line.strip().split(',')
            self.URLs[lineArr[0]] = lineArr[1]
            
    def find_most_freq_visitor(self, page_id, visitor_data):
        # find the most frequent visotor for each page.
        max_visitor = None
        max_visits = 0
        for visitor_id, visit_counts in visitor_data:
            if visit_counts > max_visits:
                max_visitor = visitor_id
                max_visits = visit_counts
        page_url = 'UNKN'
        # lookup URL
        if page_id in self.URLs:
            page_url = self.URLs[page_id]
        
        yield None, '\t'.join([page_url, page_id, max_visitor])
    
if __name__ == '__main__':
    TopVisitor.run()

Overwriting top_visitor.py


In [24]:
# local test
!python top_visitor.py reformated_webdata.csv.backup --file url_data.csv > 4_4_local_output.txt

no configs found; falling back on auto-configuration
no configs found; falling back on auto-configuration
creating tmp directory /tmp/top_visitor.root.20160206.071928.722265

PLEASE NOTE: Starting in mrjob v0.5.0, protocols will be strict by default. It's recommended you run your job with --strict-protocols or set up mrjob.conf as described at https://pythonhosted.org/mrjob/whats-new.html#ready-for-strict-protocols

writing to /tmp/top_visitor.root.20160206.071928.722265/step-0-mapper_part-00000
Counters from step 1:
  (no counters found)
writing to /tmp/top_visitor.root.20160206.071928.722265/step-0-mapper-sorted
> sort /tmp/top_visitor.root.20160206.071928.722265/step-0-mapper_part-00000
writing to /tmp/top_visitor.root.20160206.071928.722265/step-0-reducer_part-00000
Counters from step 1:
  (no counters found)
writing to /tmp/top_visitor.root.20160206.071928.722265/step-1-mapper_part-00000
Counters from step 2:
  (no counters found)
writing to /tmp/top_visitor.root.20160206.071928.7

In [25]:
# show some sample output
!cat 4_4_local_output.txt | head -50

regwiz	1000	10001
support	1001	10001
athome	1002	10001
kb	1003	10002
search	1004	10003
norge	1005	10004
misc	1006	10005
ie_intl	1007	10007
msdownload	1008	10009
windows	1009	10009
vbasic	1010	10010
officedev	1011	10010
outlookdev	1012	10010
vbasicsupport	1013	10010
officefreestuff	1014	10010
msexcel	1015	10011
excel	1016	10011
products	1017	10011
isapi	1018	10011
mspowerpoint	1019	10011
msdn	1020	10012
visualc	1021	10012
truetype	1022	10013
spain	1023	10014
iis	1024	10015
gallery	1025	10016
sitebuilder	1026	10016
intdev	1027	10017
oledev	1028	10017
clipgallerylive	1029	10019
ntserver	1030	10019
msoffice	1031	10019
games	1032	10019
logostore	1033	10019
ie	1034	10020
windowssupport	1035	10021
organizations	1036	10021
windows95	1037	10021
sbnmember	1038	10021
isp	1039	10021
office	1040	10021
workshop	1041	10021
vstudio	1042	10021
smallbiz	1043	10021
mediadev	1044	10024
netmeeting	1045	10025
iesupport	1046	10027
publisher	1048	10030
supportnet	1049	10031
macoffice	1050	10032


# HW 4.5
Here you will use a different dataset consisting of word-frequency distributions 
for 1,000 Twitter users. These Twitter users use language in very different ways,
and were classified by hand according to the criteria:

0: Human, where only basic human-human communication is observed.

1: Cyborg, where language is primarily borrowed from other sources
(e.g., jobs listings, classifieds postings, advertisements, etc...).

2: Robot, where language is formulaically derived from unrelated sources
(e.g., weather/seismology, police/fire event logs, etc...).

3: Spammer, where language is replicated to high multiplicity
(e.g., celebrity obsessions, personal promotion, etc... )

The main data lie in the accompanying file: topUsers_Apr-Jul_2014_1000-words.txt
and are of the form:
USERID,CODE,TOTAL,WORD1_COUNT,WORD2_COUNT,...
where
USERID = unique user identifier
CODE = 0/1/2/3 class code
TOTAL = sum of the word counts

Using this data, you will implement a 1000-dimensional K-means algorithm in MrJob on the users
by their 1000-dimensional word stripes/vectors using several 
centroid initializations and values of K.

Note that each "point" is a user as represented by 1000 words, and that
word-frequency distributions are generally heavy-tailed power-laws
(often called Zipf distributions), and are very rare in the larger class
of discrete, random distributions. For each user you will have to normalize
by its "TOTAL" column. Try several parameterizations and initializations:

(A) K=4 uniform random centroid-distributions over the 1000 words
(B) K=2 perturbation-centroids, randomly perturbed from the aggregated (user-wide) distribution 
(C) K=4 perturbation-centroids, randomly perturbed from the aggregated (user-wide) distribution 
(D) K=4 "trained" centroids, determined by the sums across the classes.

and iterate until a threshold (try 0.001) is reached.
After convergence, print out a summary of the classes present in each cluster.
In particular, report the composition as measured by the total
portion of each class type (0-3) contained in each cluster,
and discuss your findings and any differences in outcomes across parts A-D.

Note that you do not have to compute the aggregated distribution or the 
class-aggregated distributions, which are rows in the auxiliary file:
topUsers_Apr-Jul_2014_1000-words_summaries.txt

In [21]:
%%writefile Kmeans.py
from numpy import argmin, array, random
from mrjob.job import MRJob
from mrjob.step import MRJobStep
from itertools import chain

#Calculate find the nearest centroid for data point 
def MinDist(datapoint, centroid_points):
    datapoint = array(datapoint)
    centroid_points = array(centroid_points)
    diff = datapoint - centroid_points 
    diffsq = diff*diff
    # Get the nearest centroid for each instance
    minidx = argmin(list(diffsq.sum(axis = 1)))
    return minidx

#Check whether centroids converge
def stop_criterion(centroid_points_old, centroid_points_new,T):
    oldvalue = list(chain(*centroid_points_old))
    newvalue = list(chain(*centroid_points_new))
    Diff = [abs(x-y) for x, y in zip(oldvalue, newvalue)]
    Flag = True
    for i in Diff:
        if(i>T):
            Flag = False
            break
    return Flag

class MRKmeans(MRJob):
    centroid_points=[]
    k=4
    dim=1000
    def steps(self):
        return [
            MRJobStep(mapper_init = self.mapper_init, mapper=self.mapper,combiner = self.combiner,reducer=self.reducer)
            #MRJobStep(mapper_init = self.mapper_init, mapper=self.mapper,reducer=self.reducer)
               ]
    #load centroids info from file
    def mapper_init(self):
        self.centroid_points = [map(float,s.split('\n')[0].split(',')) for s in open("Centroids.txt").readlines()]
        self.k = len(self.centroid_points)
        self.k = len(self.centroid_points[0])
        #open('/data/w261/hw4/Centroids.txt', 'w').close()
        open('/data/w261/hw4/fitted_results.txt', 'w').close()
    #load data and output the nearest centroid index and data point 
    def mapper(self, _, line):
        lineArr = line.strip().split(',')
        user_id = lineArr[0]
        user_class = lineArr[1]
        user_total_words = float(lineArr[2])
        # normalize by user total word counts
        D = (map(lambda x: float(x)/user_total_words,lineArr[3:]))
        fitted_centroid_idx = int(MinDist(D,self.centroid_points))
        with open('/data/w261/hw4/fitted_results.txt', 'a') as f:
            f.writelines(','.join([str(fitted_centroid_idx), user_class, user_id]) + '\n')
        yield fitted_centroid_idx, ([1]+D)
    #Combine sum of data points locally
    def combiner(self, idx, inputdata):
        datasum = []
        for data in inputdata:
            if len(datasum) == 0:
                datasum = array(data)
            else:
                datasum = datasum + data
        yield idx, list(datasum)    
    #Aggregate sum for each cluster and then calculate the new centroids
    def reducer(self, idx, inputdata): 
        #print "running reducer for centroid index %s"%idx
        centroids = []
        num = [0]*self.k 
        for i in range(self.k):
            centroids.append([0]*self.dim)
        for data in inputdata:
            n = data[0]
            data = data[1:]
            num[idx] = num[idx] + n
            for data_idx, data_val in enumerate(data):
                centroids[idx][data_idx] = centroids[idx][data_idx] + data_val
        for centroid_val_idx, centroid_val in enumerate(centroids[idx]):
            centroids[idx][centroid_val_idx] = centroid_val/num[idx]
        #with open('/data/w261/hw4/Centroids.txt', 'a') as f:
        #    f.writelines(','.join(map(str,centroids[idx])) + '\n')
        yield idx,centroids[idx]
      
if __name__ == '__main__':
    MRKmeans.run()

Overwriting Kmeans.py


In [23]:
%%writefile kmeans_driver.py
from numpy import random
from random import randint
from Kmeans import MRKmeans, stop_criterion
import sys

init_options = sys.argv[1].upper()

mr_job = MRKmeans(args=['topUsers_Apr-Jul_2014_1000-words.txt', '--file',  'Centroids.txt'])

# function that prints out purity results of the fitted clusters
def printFittedResults():
    results = {}
    for line in open('fitted_results.txt', 'r'):
        line = line.strip()
        if line == '': continue
        lineArr = line.split(',')
        fitted_class = lineArr[0]
        actual_class = lineArr[1]
        if fitted_class not in results:
            results[fitted_class] = {actual_class: 1, 'total': 1}
        else:
            fitted_results = results[fitted_class]
            fitted_results['total'] += 1
            if actual_class not in fitted_results:
                fitted_results[actual_class] = 1
            else:
                fitted_results[actual_class] += 1
    for fitted_class in results:
        fitted_results = results[fitted_class]
        print '---------------------------------'
        print "Fitted Cluster Index: %s" % fitted_class
        print "\ttotal records: %s" % fitted_results['total']
        for actual_class in fitted_results:
            if actual_class != 'total':
                print "\tactual class %s: %s [%.4f]" % (actual_class, fitted_results[actual_class], float(fitted_results[actual_class])/fitted_results['total'])

#Geneate k initial centroids with uniform distribution
def generateKUniform(k, feature_size, lower_bound, upper_bound):
    results = []
    for i in range(k):
        results.append([random.uniform(0,0.5) for i in range(feature_size)])
    return results

# randomly perturb from user-wide aggregated data
def randomPerturbUserWide(k):
    summary_data = []
    results = []
    #summary_total_count = 0
    for line in open('topUsers_Apr-Jul_2014_1000-words_summaries.txt', 'r'):
        line = line.strip()
        if line.startswith('ALL_CODES,'):
            lineArr = line.split(',')
            #summary_total_count = float(lineArr[2])
            summary_data = map(float, lineArr[3:])
            break
    for i in range(k):
        perturbed_data = map(lambda x: x+random.uniform(0,10), summary_data)
        perturbed_sum = sum(perturbed_data)
        results.append(map(lambda x: x/perturbed_sum, perturbed_data))
    return results

# randomly pick k records from data as centroids
def randomPickKUsers(k):
    results = []
    data = []
    for line in open('topUsers_Apr-Jul_2014_1000-words.txt', 'r'):
        line = line.strip()
        if line == '': continue
        data.append(line)
    for i in range(k):
        selectedIdx = randint(0, len(data)-1)
        selectedLine = data[selectedIdx].split(',')
        total_words = float(selectedLine[2])
        results.append(map(lambda x: float(x)/total_words, selectedLine[3:]))
    return results

# find averages from class-wide distribution as the initilizing points
def loadTrainedCentroids():
    results = []
    for line in open('topUsers_Apr-Jul_2014_1000-words_summaries.txt', 'r'):
        line = line.strip()
        if line == '' or not line.startswith('CODE,'): continue
        lineArr = line.split(',')
        total_words = float(lineArr[2])
        results.append(map(lambda x: float(x)/total_words, lineArr[3:]))
    return results

# persist centroid points to disc
def saveCentroid():
    with open('Centroids.txt', 'w+') as f:
        f.writelines(','.join(str(j) for j in i) + '\n' for i in centroid_points)

# feature_size
dim = 1000
# convergence threahold
thold = 0.001

if init_options == 'A':
    # (A) K=4, uniform random centroids
    print "K=4, uniformly random initialization"
    centroid_points = randomPickKUsers(4)
elif init_options == 'B':
    # (B) K=2 with perturbation-centroid
    print "K=2, perturbed centroids from user-wide aggregation"
    centroid_points = randomPerturbUserWide(2)
elif init_options == 'C':
    # (C) K=4 with perturbation-centroid
    print "K=4, perturbed centroids from user-wide aggregation"
    centroid_points = randomPerturbUserWide(4)
else:
    # (D) K=4 with trained centroid
    print "Initialize centroids with training data"
    centroid_points = loadTrainedCentroids()

# save centroid initialization
saveCentroid()

# Update centroids iteratively
i = 0
while(1):
    # save previous centoids to check convergency
    centroid_points_old = centroid_points[:]
    #print "iteration"+str(i)+":"
    with mr_job.make_runner() as runner:
        runner.run()
        # stream_output: get access of the output 
        for line in runner.stream_output():
            key,value =  mr_job.parse_output_line(line)
            #print key, value
            centroid_points[key] = value
    # persist the current centroid point
    saveCentroid()
    i = i + 1
    if(stop_criterion(centroid_points_old,centroid_points,thold)):
        break
#print "Centroids\n"
#print centroid_points
print "converged at iteration: %s" % i
printFittedResults()

Overwriting kmeans_driver.py


In [24]:
!python kmeans_driver.py A

K=4, uniformly random initialization
No handlers could be found for logger "mrjob.runner"
converged at iteration: 10
---------------------------------
Fitted Cluster Index: 1
	total records: 130
	actual class 1: 88 [0.6769]
	actual class 3: 3 [0.0231]
	actual class 2: 38 [0.2923]
	actual class 0: 1 [0.0077]
---------------------------------
Fitted Cluster Index: 0
	total records: 60
	actual class 0: 2 [0.0333]
	actual class 3: 58 [0.9667]
---------------------------------
Fitted Cluster Index: 3
	total records: 108
	actual class 0: 82 [0.7593]
	actual class 1: 1 [0.0093]
	actual class 2: 15 [0.1389]
	actual class 3: 10 [0.0926]
---------------------------------
Fitted Cluster Index: 2
	total records: 702
	actual class 1: 2 [0.0028]
	actual class 0: 667 [0.9501]
	actual class 3: 32 [0.0456]
	actual class 2: 1 [0.0014]


In [25]:
!python kmeans_driver.py B

K=2, perturbed centroids from user-wide aggregation
No handlers could be found for logger "mrjob.runner"
converged at iteration: 5
---------------------------------
Fitted Cluster Index: 1
	total records: 133
	actual class 1: 88 [0.6617]
	actual class 3: 4 [0.0301]
	actual class 2: 40 [0.3008]
	actual class 0: 1 [0.0075]
---------------------------------
Fitted Cluster Index: 0
	total records: 867
	actual class 0: 751 [0.8662]
	actual class 3: 99 [0.1142]
	actual class 2: 14 [0.0161]
	actual class 1: 3 [0.0035]


In [26]:
!python kmeans_driver.py C

K=4, perturbed centroids from user-wide aggregation
No handlers could be found for logger "mrjob.runner"
converged at iteration: 6
---------------------------------
Fitted Cluster Index: 1
	total records: 578
	actual class 0: 512 [0.8858]
	actual class 3: 66 [0.1142]
---------------------------------
Fitted Cluster Index: 0
	total records: 130
	actual class 1: 88 [0.6769]
	actual class 3: 4 [0.0308]
	actual class 2: 37 [0.2846]
	actual class 0: 1 [0.0077]
---------------------------------
Fitted Cluster Index: 3
	total records: 182
	actual class 0: 162 [0.8901]
	actual class 1: 3 [0.0165]
	actual class 2: 6 [0.0330]
	actual class 3: 11 [0.0604]
---------------------------------
Fitted Cluster Index: 2
	total records: 110
	actual class 0: 77 [0.7000]
	actual class 3: 22 [0.2000]
	actual class 2: 11 [0.1000]


In [22]:
!python kmeans_driver.py D

Initialize centroids with training data
No handlers could be found for logger "mrjob.runner"
converged at iteration: 5
---------------------------------
Fitted Cluster Index: 1
	total records: 51
	actual class 1: 51 [1.0000]
---------------------------------
Fitted Cluster Index: 0
	total records: 804
	actual class 0: 749 [0.9316]
	actual class 3: 38 [0.0473]
	actual class 2: 14 [0.0174]
	actual class 1: 3 [0.0037]
---------------------------------
Fitted Cluster Index: 3
	total records: 63
	actual class 0: 2 [0.0317]
	actual class 3: 61 [0.9683]
---------------------------------
Fitted Cluster Index: 2
	total records: 82
	actual class 1: 37 [0.4512]
	actual class 3: 4 [0.0488]
	actual class 2: 40 [0.4878]
	actual class 0: 1 [0.0122]


<b>General Discussion on 4.5</b>

Initialization with trained data obtained the result with highest purity and least number of iterations.  When using user-wide perturbation and K=4, the result is less pure but also didn't take many iterations.  K=4 uniformly pick from user records requires the longest iteration times.  With K=2, the results is not very pure, which is not surprising since we already know the there are four true classes.